# Uso avanzado de RDDs

En Big Data é habitual traballar con datos en formato **clave–valor**.  
Por este motivo, Spark ofrece transformacións e accións específicas deseñadas para operar eficientemente con este tipo de datos.


In [1]:
# Inicializamos SparkSession y SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("02-rdd2") \
    .getOrCreate()

sc = spark.sparkContext
print(spark.version)  # Verifica la versión de Spark




:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1583e940-cd56-479d-8587-2537b6c2d9e7;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

3.5.7


In [ ]:
# Exemplo: creamos RDD de pares a partir de lista de tuplas
rdd_pares1 = sc.parallelize([('a', 1), ('b', 1), ('c', 1)])
print ("RDD de pares 1: ",rdd_pares1.collect())

# Exemplo 2: creamos RDD de pares a partir dun RDD simple con map
rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())
rdd_pares2 = rdd_st.map(lambda palabra: (palabra,1))
print ("RDD de pares 2: ",rdd_pares2.collect())

> O interesante dos RDD's de pares clave-valor é que presentan unha serie de transformacións e accións adicionais.

## Operacións clave-valor básicas
### keyBy
Transformación que converte un RDD de elementos nun RDD de pares (clave, valor), xerando a clave a partir de cada elemento mediante unha función.

In [ ]:
# Exemplo 1: A clave é a inicial de cada palabra
rdd_pares = rdd_st.keyBy(lambda palabra: palabra[0])
print("Palabras por inicial: ",rdd_pares.collect())

# Exemplo 2: A clave é a lonxitude da palabra
rdd_pares2 = rdd_st.keyBy (lambda palabra: len(palabra))
print("Palabras por lonxitude: ",rdd_pares2.collect())

# Exemplo 3: A clave é o feito de se o elemento é par ou non
rdd_int = sc.parallelize(range(10))
rdd_pares3 = rdd_int.keyBy(lambda x: x%2 )
print ("Números por ser par ou non: ", rdd_pares3.collect())

`keyBy` é moi útil para preparar RDD's para as seguintes operacións:
- `groupByKey`
- `reduceByKey`
- `countByKey`
- `join`
- `partitionBy`

Xa que estas operacións só funcionan con RDD's tipo (k,v).

### mapValues
Realiza una operación map só sobre os valores do RDD, deixando a clave tal como está.

In [ ]:
# Exemplo 1: pasomos o valores a maiúsculas, sen tocar a clave
print ( "Palabras por inicial en maiúsculas: ", rdd_pares.mapValues( lambda x: x.upper()).collect())

# Exemplo 2: Operación matemática sobre os valores sen modificar as claves
rdd_x10 = rdd_pares3.mapValues(lambda x: x*10)
print ("Números multiplicados por 10: ", rdd_x10.collect())


`mapValues` é util cando:
- Temos unha clave que nos interesa conservar.
- Queremos facer operacións sobre os valores.

**Exemplos reais**:
- Despois dun `reduceByKey`, para escalar ou normalizar os resultados.
- Trasformar listas, estatísticas ou estruturas asociadas a unha clave.
- Prepara datos para un join mantendo unha clave común.

**Diferencia con map**
- A función especificada en `map` aplícase tanto á clave como ao valor:



In [ ]:
# Exemplo: pasamos todo a maiúsculas
print ( "Palabras por inicial en maiúsculas: ", rdd_pares.map( lambda par: (par[0].upper(), par[1].upper())).collect())

### keys
Transformación que devolve un RDD formado só polas claves dun RDD de pares (K, V), eliminando os valores.

In [ ]:
# Exemplo: Creamos un RDD só coas claves
print("RDD de claves: ", rdd_pares.keys().collect())

`keys()` é útil cando queres traballar só coas claves, por exemplo:
- ver que tipos de claves hai
- facer un `distinct()` para obter claves únicas
- contar cantas claves hai ou analizar a súa distribución
- cruzalo con outro RDD ou facer operacións de filtrado

En realidade, pódese facer unha transformación equivalente empregando `map`.

In [ ]:
print("Só claves (map): ", rdd_pares.map(lambda par: par[0]).collect())

### values
Transformación que devolve un RDD formado só polos valores dun RDD de pares (K, V), eliminando as claves.

In [ ]:
# Exemplo: Creamos un RDD formado só polos valores
print("RDD de valores: ",rdd_pares.values().collect())

`values` é útil cando:
- só interesa o dato asociado á clave.
- quérense facer operacións estatísticas sobre os valores.

En realidade, ao igual que con `keys`, pódese facer unha transformación equivalente empregando `map`.

In [ ]:
# Exemplo: qudearse só cos valores mediante map
print("Só valores (map): ", rdd_pares.map(lambda par: par[1]).collect())

### lookup
`lookup()` é unha acción dispoñible só para **Pair RDDs** (RDDs de pares `(K, V)`), que permite recuperar todos os valores asociados a unha clave concreta.

- **Entrada:** un `RDD[(K, V)]`
- **Saída:** unha lista de valores `List[V]` (en PySpark, unha lista Python)
- **Comportamento:** devolve *tódolos valores* que teñan exactamente esa clave (poden ser varios)

**Transformación conceptual:**

`RDD[(K, V)] → lookup(k) → List[V]`


In [ ]:
# Exemplo: obtemos todos os elementos coa clave "B"
print("Palabras que empezan por B: ",rdd_pares.lookup('B'))

`lookup` é útil:
- Cando se quere obter rapidamente os valores dun elemento concreto sen facer un `filter()` + `collect()`.
### sampleByKey
`sampleByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que permite obter unha mostra (sample) dos elementos, pero **facendo a mostra por clave**.

É dicir: en lugar de mostrear todo o RDD ao azar, **Spark aplica unha fracción de mostra diferente para cada clave**, mantendo así un control por grupos.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** outro `RDD[(K, V)]` cun subconxunto dos pares orixinais
- **Idea:** “Coller unha mostra estratificada por clave”

---

#### Parámetros
- `withReplacement` (`bool`)
  - `False`: selección sen reposición (un elemento só pode aparecer unha vez)
  - `True`: selección con reposición (pode repetirse)
- `fractions` (`dict`)
  - Un dicionario co formato `{clave: fracción}`
  - A fracción indica a proporción aproximada de elementos que se seleccionan para esa clave (0.0 → 0%, 1.0 → 100%)
- `seed` (opcional)
  - Semente para reproducibilidade no proceso aleatorio

---


In [ ]:
# Exemplo: mostras proporcionadas.

rdd = sc.parallelize([
    ("A", 1), ("A", 2), ("A", 3), ("A", 4),
    ("B", 10), ("B", 20), ("B", 30),
    ("C", 100), ("C", 200)
])

# De A collemos apróximadamente 0 50% dos elementos. De B todos (100%) e d e C ningún (0%).
fractions = {"A": 0.5, "B": 1.0, "C": 0.0}

sample = rdd.sampleByKey(withReplacement=False, fractions=fractions, seed=42)

print(sample.collect())

`sampleByKey` server para:
- facer mostras mantendo a proporción por grupos/claves
- equilibrar datasets (ex.: coller menos exemplos dunha clase maioritaria)
- crear subconxuntos para test/validación mantendo distribución por clave

## Agregacións
### groupByKey
`groupByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que agrupa todos os valores que teñen a mesma clave, devolvendo un RDD onde cada clave queda asociada a unha colección (iterable) de valores.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** `RDD[(K, Iterable[V])]`
- **Idea:** “Para cada clave, xunta todos os valores nunha lista/iterable”

**Transformación conceptual**:
`RDD[(K, V)] → RDD[(K, Iterable[V])]`

In [ ]:
# Exemplo: Agrupamos palabras por inicial:
for key, values_iterable in rdd_pares.groupByKey().collect():
    # Convertir el iterable de resultados a una lista
    values_list = list(values_iterable)
    # Imprimir la clave y los valores
    print(f"Clave: {key}, Valores: {values_list}")

`groupByKey` é útil cando se necesita:
- Obter todos os valores dunha clave e procesalos como xunto.
- Aplicar operacións complexas por grupo.

**Advertencia importante (rendemento)**

En moitos casos `groupByKey()` **non é a mellor opción**, porque:

- obriga a mover moitos datos pola rede (**shuffle**)
- garda todos os valores dunha clave na memoria (pode causar **OutOfMemoryError**)

Se o que se quere é **agregar** (sumar, contar, máximo, mínimo...), é mellor usar:

- `reduceByKey()`
- `aggregateByKey()`
- `combineByKey()`
- 
### reduceByKey
`reduceByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que combina os valores que comparten a mesma clave aplicando unha función de redución (por exemplo suma, máximo, concatenación, etc.).

A diferenza de `groupByKey()`, **non crea unha lista con todos os valores**, senón que os vai combinando de forma incremental, o que o fai moito máis eficiente.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** `RDD[(K, V)]`
- **Requisito:** a función debe ser do tipo `(V, V) → V` (mesmo tipo de entrada e saída)

**Transformación conceptual**:

`RDD[(K, V)] → RDD[(K, V)]`


In [ ]:
rdd_pares.reduceByKey(lambda x,y:len(x)+len(y)).collect()

`reduceBykey` serve para:
- sumar valores por clave (contaxes, totais…)
- obter máximos ou mínimos por clave
- combinar valores reducindo (p.ex. concatenar cadeas)
- facer agregacións eficientes a gran escala

> Sempre que a operación sexa reducible, débese priorizar `reduceByKey()` fronte a `groupByKey()`.

### sortByKey
`sortByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que ordena os elementos do RDD **segundo a clave**.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** `RDD[(K, V)]` cos pares ordenados pola clave
- **Idea:** “Ordenar o RDD polo primeiro elemento do par (a clave)”

📌 Transformación conceptual:
`RDD[(K, V)] → RDD[(K, V)] (ordenado por K)`

#### Parámetros:
- `ascending`: orde ascendente ou descentente (por defecto `True`).
- `nunPartitions`: número de particións do RDD resultante (útil para controlar o paralelismo).
- `keyfunc`: Permite aplicar unha función á clave antes de ordenar.

**Recoméndase usar**:
- só cando realmente se precise ordenación total
- con datasets xa reducidos/agruapados.

In [ ]:
# Exemplo: Ordenar por clave descendentemente
print("Por clave descendente: ", rdd_pares3.sortByKey(ascending=False).collect())

### countByKey
`countByKey()` é unha **acción** dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que conta cantos elementos hai para cada clave, devolvendo un dicionario co número de aparicións de cada clave.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** `dict` (en Python) co formato `{clave: número_de_elementos}`
- **Idea:** “Cantas veces aparece cada clave?”

Transformación conceptual:

`RDD[(K, V)] → dict(K → count)`


In [ ]:
# Exemplo: contar as ocorrencias de cada clave

rdd_pares.countByKey()

`countByKey` é útil cando se quere:
- Saber a distribución de elementos por clave.
- Facer unha análise rápida de claves.
- Validar datos (ver se hai claves con poucos elementos).

**Advertencia**
Como `countByKey` é unha acción, trae o resultado ao *driver*. Se o dicionario é demasiado grande pode xerar problemas de memoria.

### aggregate
`aggregate()` é unha **acción** que permite reducir un RDD a un único resultado, aplicando:

- unha función para combinar elementos dentro de cada partición (`seqOp`)
- outra función para combinar resultados entre particións (`combOp`)
- partindo dun valor inicial (`zeroValue`)

É máis flexible que `reduce()`, porque:
- o valor acumulado pode ser dun tipo distinto ao do RDD
- permite definir unha lóxica diferente para o cálculo local e global

#### Parámetros
- `zeroValue`: valor inicial do acumulador
- `seqOp(acc, x)`: como se acumula un elemento `x` no acumulador dentro dunha partición
- `combOp(acc1, acc2)`: como se combinan acumuladores de distintas particións



In [ ]:
# Exemplo: cálculo da media
rdd = sc.parallelize([10, 20, 30, 40])

# Establecemos o valor inicial
zero = (0, 0)  # (suma, conta)

# Definimos a operación de acumulación
seqOp = lambda acc, x: (acc[0] + x, acc[1] + 1)

# Definimos a operación de combinación
combOp = lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])

# Obtemos a suma total e o número de elementos
suma, conta = rdd.aggregate(zero, seqOp, combOp)

# Xa no driver, calculamos e amosamos a media.
media = suma / conta
print(media)

In [ ]:
# Exemplo: Cálculo de máximo e suma simultáneamente

# Creamos un RDD con 4 elementos e 2 particións
rdd_pruebas = sc.parallelize([1, 2, 3, 4], 2)

# glom() agrupa os elementos de cada partición nunha lista
# collect() tráeo ao driver para poder velo
print(rdd_pruebas.glom().collect())

# seqOp (función de secuencia): combína un acumulador coa seguinte entrada do RDD
# acumulador x = (suma_parcial, contador_parcial)
# elemento y = valor do RDD
seqOp = lambda x, y: (x[0] + y, x[1] + 1)

# combOp (función de combinación): combina dous acumuladores (de particións distintas)
# x e y son acumuladores: (suma, contador)
combOp = lambda x, y: (x[0] + y[0], x[1] + y[1])

# zero é o valor inicial
zero = (0,0)

# aggregate(zeroValue, seqOp, combOp)
# zeroValue = (0,0) é o acumulador inicial
# Resultado: (suma_total, conta_total)
resultado = rdd_pruebas.aggregate(zero, seqOp, combOp)

print("(suma, conteo): ", resultado)


In [ ]:
# Exemplo: cálculo do produto e do número de elementos

# Definimos a seqOp
# calcúlanse pares coa seguinte forma (produto_parcial, contador_parcial)
seqOp = (lambda x,y: (x[0] * y, x[1]+1))

# Definimos a CombOp
# Multiplicamos os produtos e sumamos os contadores
combO = (lambda x,y: (x[0] * y[0], x[1]+y[1]))

# Definimos o zero
# Hai que ter en conta que o valor neutro do produto e o 1.
zero = (1,0)

# Calculamos o resultado final
mult, count = rdd_pruebas.aggregate(zero,seqOp, combOp)
print ("Produto: ", mult,", Conteo: ",count)

**`aggregate` úsase...**
- Cando s necesita máis dunha métrica.
- Cando se pretende empregar acumuladores máis complexos.
- Cando `reduceByKey` non é suficiente, xa que o resultado ten que ser do mesmo tipo que o orixinal.

> É moito máis eficiente que usar `gropByKey` + cálculo

### aggregateByKey
aggregateByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que permite combinar os valores asociados a cada clave usando un acumulador, de forma similar a `aggregate()`, pero **aplicado por clave**.

- **Entrada:** `RDD[(K, V)]`
- **Saída:** `RDD[(K, U)]` onde `U` é o tipo do acumulador
- **Idea:** “Para cada clave, acumular e combinar valores usando un valor inicial e dúas funcións (local e global)”

📌 Transformación conceptual:
`RDD[(K, V)] → RDD[(K, U)]`

---

#### Parámetros
- `zeroValue`
  - valor inicial do acumulador **para cada clave** en cada partición
  - debe ser un valor neutro (por exemplo `(0,0)` para suma+conta)
- `seqOp(acc, v)`
  - combina o acumulador `acc` cun valor `v` dentro dunha partición
- `combOp(acc1, acc2)`
  - combina dous acumuladores parciais (de particións distintas)


In [ ]:
# Exemplo: suma e conteo por clave

# Creamos un RDD de pares (k,v)
rdd = sc.parallelize([
    ("a", 10), ("a", 20),
    ("b", 5), ("b", 15), ("b", 10)
])

# Definimos o valor zero
zero = (0, 0)  # (suma, conta)

# Definimos a SecOp
seqOp = lambda acc, v: (acc[0] + v, acc[1] + 1)

# Definimos a combOp
combOp = lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])

# Obtemos e amosamos o resultado
res = rdd.aggregateByKey(zero, seqOp, combOp)

print("Resultado: ",res.collect())

# Calculamos as medias:
medias = res.mapValues(lambda sc: sc[0] /sc[1])
print("Medias: ",medias.collect())


**`aggregateByKey` úsase...**
- Cando se necesita obter varias métricas por clave.
- Cando se necesitan acumuladores complexos (tuplas, listas, conxuntos...).
- Cando `reduceByKey` non é suficiente.

### combineByKey
`combineByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que permite crear e combinar acumuladores por clave de forma totalmente flexible.

É a operación máis xeral de agregación por clave en Spark e serve de base para outras funcións como:
- `reduceByKey()`
- `aggregateByKey()`

**Transformación conceptual**:

`RDD[(K, V)] → RDD[(K, C)]`  
onde `C` é o tipo do acumulador (combiner).

---

#### Como funciona?
Para cada clave, Spark:
1. **Crea un acumulador inicial** a partir do primeiro valor que aparece para esa clave (`createCombiner`)
2. Vai engadindo novos valores a ese acumulador dentro da mesma partición (`mergeValue`)
3. Combina acumuladores parciais de particións distintas (`mergeCombiners`)

---

#### Parámetros
- `createCombiner(v)`
  - transforma un primeiro valor `v` nun acumulador inicial
- `mergeValue(acc, v)`
  - engade un valor `v` ao acumulador `acc` dentro da mesma partición
- `mergeCombiners(acc1, acc2)`
  - combina dous acumuladores procedentes de particións diferentes

---

In [ ]:
# Exemplo: Calcular a media de valores por clave usando combineByKey (sen groupByKey)

# Creamos un RDD de pares (k, v) con varias entradas para as claves "a" e "b"
rdd = sc.parallelize([
    ("a", 10), ("a", 20),
    ("b", 5), ("b", 15), ("b", 10)
])

# Definimos como crear o acumulador inicial a partir do primeiro valor dunha clave
# Neste caso o acumulador será unha tupla (suma, conta)
createCombiner = lambda v: (v, 1)

# Definimos como engadir un novo valor ao acumulador dentro da mesma partición
# Engadimos o valor á suma e incrementamos o contador
mergeValue = lambda acc, v: (acc[0] + v, acc[1] + 1)

# Definimos como combinar acumuladores de particións distintas (durante o shuffle)
# Sumamos as sumas parciais e tamén os contadores parciais
mergeCombiners = lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])

# Aplicamos combineByKey para obter, para cada clave, a tupla (suma_total, conta_total)
sum_count = rdd.combineByKey(createCombiner, mergeValue, mergeCombiners)

# Calculamos a media por clave dividindo suma_total entre conta_total
medias = sum_count.mapValues(lambda sc: sc[0] / sc[1])

# Amosamos o resultado final: media dos valores asociados a cada clave
print("Medias por clave: ", medias.collect())


**`combineByKey` úsase...**
- Cando se necesita control total sobre como se crea e combina o acumulador.
- Cando se quere transformar o tipo de acumuladro.
- Cando se queren implementar agregacións avanzadas.

#### Por que é importante?

`combineByKey()` é a función máis potente para agregación por clave.  
As demais son versións simplificadas:

##### `reduceByKey(func)` equivale a:
```python
createCombiner = lambda v: v
mergeValue = func
mergeCombiners = func
```
##### `aggregateByKey(func)` equivale a:
```python
createCombiner = lambda v: seqOp(zero, v)
mergeValue = seqOp
mergeCombiners = combOp
```

### foldByKey
`foldByKey()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que combina os valores asociados a cada clave aplicando unha función de redución, partindo dun valor inicial común (`zeroValue`).

É moi semellante a `reduceByKey()`, pero coa diferenza de que:
- usa un **valor inicial (`zeroValue`)**
- este valor inicial debe ser **neutro** respecto da función de combinación
- a función `func` debe ser do tipo `(V, V) -> V` (mesmo tipo de entrada e saída)

📌 Transformación conceptual:
`RDD[(K, V)] → RDD[(K, V)]`

---

#### Parámetros
- `zeroValue`
  - valor inicial que se aplica para cada clave en cada partición
  - debe ser un elemento neutro (por exemplo `0` para suma, `1` para multiplicación)
- `func(x, y)`
  - función que combina dous valores do mesmo tipo
  - debe ser asociativa e conmutativa para garantir resultados consistentes

---

In [ ]:
# Exemplo: Sumar os valores por clave usando foldByKey cun valor inicial neutro (0)

# Creamos un RDD de pares (k, v) con varias entradas por clave
rdd = sc.parallelize([
    ("a", 10), ("a", 20),
    ("b", 5), ("b", 15), ("b", 10)
])

# Aplicamos foldByKey usando 0 como valor inicial (neutro para a suma)
# A función combina os valores asociados á mesma clave sumándoos
resultado = rdd.foldByKey(0, lambda x, y: x + y)

# Amosamos o resultado final: suma total dos valores por clave
print("Suma por clave: ", resultado.collect())


In [ ]:
# Exemplo: Calcular o produto dos valores por clave usando foldByKey cun valor inicial neutro (1)

# Creamos un RDD de pares (k, v) con varias entradas por clave
rdd = sc.parallelize([
    ("a", 2), ("a", 3),
    ("b", 4), ("b", 5)
])

# Aplicamos foldByKey usando 1 como valor inicial (neutro para o produto)
# A función combina os valores asociados á mesma clave multiplicándoos
resultado = rdd.foldByKey(1, lambda x, y: x * y)

# Amosamos o resultado final: produto total dos valores por clave
print("Produto por clave: ", resultado.collect())


**`foldByKey` úsase...**
- Cando se quere facer unha redución por clave que precisa un valor inicial.
- Cando o operador ten un elemento neutro natural.
- Cando se quere unha alternativa máis explícita a `reduceByKey`.

< **Advertencia**: Se o `zeroValue` non é neutro aplícase en cada partición.

### coGroup
`cogroup()` é unha transformación dispoñible para **Pair RDDs** (RDDs de pares `(K, V)`) que permite agrupar dous (ou máis) RDDs **pola mesma clave**, devolvendo para cada clave unha parella de coleccións:  
- os valores desa clave no primeiro RDD
- os valores desa clave no segundo RDD

É útil para comparar ou combinar datos por clave cando **non se quere facer un `join` directo**, xa que `cogroup()` conserva **tódolos valores** en cada lado.

- **Entrada:** `RDD[(K, V)]` e outro `RDD[(K, W)]`
- **Saída:** `RDD[(K, (Iterable[V], Iterable[W]))]`

📌 Transformación conceptual:
`RDD[(K, V)]` + `RDD[(K, W)] → RDD[(K, (Iterable[V], Iterable[W]))]`

---


In [ ]:
# Exemplo: Agrupar dous RDDs pola mesma clave usando cogroup

# Creamos o primeiro RDD de pares (k, v)
rdd1 = sc.parallelize([
    ("a", 1), ("b", 2), ("a", 3)
])

# Creamos o segundo RDD de pares (k, w)
rdd2 = sc.parallelize([
    ("a", 10), ("b", 20), ("b", 30)
])

# Aplicamos cogroup para agrupar por clave os valores de ambos RDDs
resultado = rdd1.cogroup(rdd2)

# Convertimos os iterables a listas só para poder visualizar mellor o contido
resultado_listas = resultado.mapValues(lambda x: (list(x[0]), list(x[1])))

# Amosamos o resultado final: para cada clave, lista de valores do rdd1 e lista de valores do rdd2
print("Valores agrupados por clave en ambos RDDs: ", resultado_listas.collect())

In [ ]:
import random
distinctChars = rdd_st.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))
resultado = charRDD.cogroup(charRDD2).take(5)

for clave, valores in resultado:
    print("Clave:", clave)
    print("Valores de charRDD:", list(valores[0]))  # Convertimos el iterable de valores a lista
    print("Valores de charRDD2:", list(valores[1])) # Convertimos el iterable de valores a lista

**`cogroup()` é útil cando se quere**
- agrupar dous datasets pola mesma clave mantendo todos os valores de cada lado
- comparar listas de valores por clave (por exemplo, elementos presentes nun dataset e non noutro)
- preparar operacións personalizadas onde un join non encaixa ben

## Joins en Pair RDDs (RDDs de pares)

En Spark existen varios tipos de **join** para combinar dous **Pair RDDs** (`RDD[(K, V)]`) usando a **clave K** como criterio de unión.

Todos os joins devolven un RDD onde cada elemento ten a forma:
`(K, (V, W))` (ou variantes con `None` no caso de outer joins)

---

### 1) `join()` — Inner join
Combina só as claves que aparecen en **ambos RDDs**.

- **Entrada:** `RDD[(K, V)]` e `RDD[(K, W)]`
- **Saída:** `RDD[(K, (V, W))]`
- Só devolve claves comúns.

> Útil cando se quere a intersección de claves.

---

### 2) `fullOuterJoin()` — Full outer join
Combina claves de **ambos RDDs**, mesmo se só aparecen nun deles.

- **Saída:** `RDD[(K, (Optional[V], Optional[W]))]`
- Se falta un valor nun lado, aparece como `None`.

> Útil cando se quere manter toda a información posible.

---

### 3) `leftOuterJoin()` — Left outer join
Mantén todas as claves do **RDD esquerdo**, e só combina as que existan no dereito.

- **Saída:** `RDD[(K, (V, Optional[W]))]`
- Se unha clave non existe no dereito, o valor dereito será `None`.

>  Útil cando se quere manter todo o dataset principal (esquerdo).

---

### 4) `rightOuterJoin()` — Right outer join
Mantén todas as claves do **RDD dereito**, e só combina as que existan no esquerdo.

- **Saída:** `RDD[(K, (Optional[V], W))]`
- Se unha clave non existe no esquerdo, o valor esquerdo será `None`.

>  Útil cando o dataset principal é o dereito.

---

### 5) `cartesian()` — Produto cartesiano (non recomendado)
Xera todas as combinacións posibles entre os elementos de dous RDDs:

- **Entrada:** `RDD[A]` e `RDD[B]`
- **Saída:** `RDD[(A, B)]`
- Non usa claves: combina todo con todo.

**Non se recomenda o seu uso**, porque:
- pode xerar un número enorme de combinacións (`n * m`)
- require moitos recursos e pode facer fallar o proceso por falta de memoria

> Só ten sentido en casos moi controlados ou con datasets moi pequenos.

---

In [ ]:
# Exemplo: Comparar distintos tipos de join entre dous Pair RDDs (inner, outer e cartesian)

# Creamos o primeiro RDD de pares (k, v) que actuará como RDD esquerdo nos joins
rdd1 = sc.parallelize([('a', 1), ('b', 2), ('c', 3), ('d', 4)])

# Creamos o segundo RDD de pares (k, v) que actuará como RDD dereito nos joins
rdd2 = sc.parallelize([('a', 4), ('b', 5), ('c', 6), ('e', 5)])

# Realizamos un inner join: devolve só as claves que existen en ambos RDDs ("a", "b" e "c")
rdd3 = rdd1.join(rdd2)

# Amosamos o resultado do inner join: (clave, (valor_rdd1, valor_rdd2))
print("Inner join:")
print("Resultado do inner join: ", rdd3.collect())


# Realizamos un full outer join: devolve todas as claves de ambos RDDs ("a", "b", "c", "d" e "e")
# Se unha clave non existe nun dos lados, o valor aparece como None
rdd4 = rdd1.fullOuterJoin(rdd2)

# Amosamos o resultado do full outer join: (clave, (valor_ou_None, valor_ou_None))
print("Full Outer join:")
print("Resultado do full outer join: ", rdd4.collect())


# Realizamos un left outer join: devolve todas as claves do RDD esquerdo (rdd1)
# Para claves que non existen no dereito, o valor dereito será None (por exemplo, "d")
rdd5 = rdd1.leftOuterJoin(rdd2)

# Amosamos o resultado do left outer join: (clave, (valor_rdd1, valor_rdd2_ou_None))
print("Left Outer join:")
print("Resultado do left outer join: ", rdd5.collect())


# Realizamos un right outer join: devolve todas as claves do RDD dereito (rdd2)
# Para claves que non existen no esquerdo, o valor esquerdo será None (por exemplo, "e")
rdd6 = rdd1.rightOuterJoin(rdd2)

# Amosamos o resultado do right outer join: (clave, (valor_rdd1_ou_None, valor_rdd2))
print("Right Outer join:")
print("Resultado do right outer join: ", rdd6.collect())


# Realizamos un produto cartesiano: combina todos os elementos de rdd1 con todos os elementos de rdd2
# O resultado terá tamaño len(rdd1) * len(rdd2) (neste caso 4 * 4 = 16 pares)
# Non se recomenda para datasets grandes porque pode xerar un número enorme de combinacións
rdd7 = rdd1.cartesian(rdd2)

# Amosamos o resultado do cartesian: ((k1, v1), (k2, v2))
print("Cartesian:")
print("Resultado do produto cartesiano: ", rdd7.collect())


### union
`union()` é unha transformación que permite unir dous RDDs nun único RDD, concatenando os seus elementos.

- **Entrada:** dous RDDs do mesmo tipo (`RDD[T]` + `RDD[T]`)
- **Saída:** un novo `RDD[T]` con todos os elementos dos dous RDDs
- **Comportamento:** non elimina duplicados, simplemente xunta contidos

📌 Transformación conceptual:
`RDD[T] + RDD[T] → RDD[T]`

---

In [ ]:
# Exemplo: Unir dous RDDs de números usando union()

# Creamos o primeiro RDD de números
rdd1 = sc.parallelize([1, 2, 3])

# Creamos o segundo RDD de números
rdd2 = sc.parallelize([3, 4, 5])

# Aplicamos union para unir ambos RDDs
resultado = rdd1.union(rdd2)

# Amosamos o resultado final: contén todos os elementos dos dous RDDs (incluíndo duplicados)
print("Unión dos dous RDDs: ", resultado.collect())

In [ ]:
# Exemplo: Unir dous Pair RDDs usando union()

# Creamos o primeiro Pair RDD
rdd1 = sc.parallelize([("a", 1), ("b", 2)])

# Creamos o segundo Pair RDD
rdd2 = sc.parallelize([("b", 3), ("c", 4)])

# Aplicamos union para unir ambos Pair RDDs
resultado_pair = rdd1.union(rdd2)

# Amosamos o resultado final: contén todos os pares, incluíndo claves repetidas
print("Unión de dous Pair RDDs: ", resultado_pair.collect())


**`union()` é útil cando se quere**:
- combinar datasets que teñen o mesmo esquema/tipo
- xuntar datos de varias fontes
- construír un dataset maior a partir de varios RDDs

> `union` non elimina duplicados. Para facelo pode empregarse `distinct`

In [ ]:
print("Union de dous RDD sen duplicados: ", resultado.distinct().collect())

### zip
`zip()` é unha transformación que permite combinar dous RDDs elemento a elemento, formando pares cos elementos da mesma posición.

- **Entrada:** dous RDDs `RDD[A]` e `RDD[B]`
- **Saída:** `RDD[(A, B)]`
- **Comportamento:** o elemento `i` do primeiro RDD combínase co elemento `i` do segundo RDD

**Transformación conceptual**:

`RDD[A] + RDD[B] → RDD[(A, B)]`

---

#### Requisitos importantes
Para que `zip()` funcione, ambos RDDs deben cumprir:

- ter o **mesmo número de elementos**
- ter o **mesmo número de particións**
- ter o **mesmo número de elementos en cada partición**

> Se non se cumpre algún destes requisitos, Spark lanza un erro.


In [ ]:
# Exemplo: Combinar dous RDDs elemento a elemento usando zip()

# Creamos o primeiro RDD de números
rdd1 = sc.parallelize([1, 2, 3, 4], 2)

# Creamos o segundo RDD de letras co mesmo número de elementos e particións
rdd2 = sc.parallelize(["a", "b", "c", "d"], 2)

# Aplicamos zip para combinar elemento a elemento
resultado = rdd1.zip(rdd2)

# Amosamos o resultado final: cada número queda emparellado coa letra da mesma posición
print("RDD combinado con zip(): ", resultado.collect())

## Controlando particións en RDDs

A API de RDDs permite controlar como se distribúen fisicamente os datos a través do clúster. O número de particións inflúe directamente no paralelismo e no rendemento: pode axudar a repartir mellor a carga de traballo, pero tamén pode provocar sobrecarga se hai demasiadas particións ou custes elevados de shuffle se hai demasiada redistribución.

### `coalesce(numPartitions, shuffle=False)`
`coalesce()` permite reducir o número de particións colapsando particións existentes, normalmente sen mover datos entre nodos. Por defecto non realiza shuffle (`shuffle=False`), polo que é unha opción eficiente cando se quere diminuír particións despois de filtros ou operacións que deixan poucos datos. Se se usa `shuffle=True`, entón si pode redistribuír datos e o custo aumenta.

In [ ]:
# Exemplo: Reducir o número de particións dun RDD usando coalesce()

rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split(), 4)

print("Número inicial de particións: ", rdd_st.getNumPartitions())

# Reducimos o número de particións a 2 sen shuffle (por defecto)
rdd2 = rdd_st.coalesce(2)

# Amosamos o número final de particións
print("Número de particións tras coalesce(2): ", rdd2.getNumPartitions())

### `repartition(numPartitions)`
`repartition()` permite modificar o número de particións dun RDD, pero sempre realiza un shuffle completo. Isto implica mover datos entre nodos para equilibrar e repartir de novo a carga. É útil cando se quere aumentar o paralelismo ou cando as particións quedaron descompensadas, pero é máis custoso que `coalesce()`.


In [ ]:
# Exemplo: Modificar o número de particións dun RDD usando repartition()

# Supoñemos un RDD chamado rdd_st
rdd4 = rdd_st.repartition(4)

# Amosamos o número final de particións
print("Número de particións tras repartition(4): ", rdd4.getNumPartitions())


### `repartitionAndSortWithinPartitions(numPartitions=None, partitionFunc=None, ascending=True, keyfunc=None)`
`repartitionAndSortWithinPartitions()` permite reparticionar os datos (shuffle) e, ademais, ordenar os elementos dentro de cada partición segundo a clave. Isto resulta útil cando se quere un reparto controlado por clave e se pretende preparar o dataset para operacións posteriores que se benefician de datos xa ordenados dentro de cada partición.

Parámetros:
- `numPartitions` (opcional): número de particións do resultado.
- `partitionFunc` (opcional): función que determina o índice da partición a partir da clave.
- `ascending` (opcional): ordenación ascendente (`True`, por defecto) ou descendente (`False`).
- `keyfunc` (opcional): función para transformar a clave antes de ordenar.



In [ ]:

# Exemplo: Reparticionar e ordenar dentro de cada partición usando repartitionAndSortWithinPartitions()

# Creamos un Pair RDD (clave, valor)
rdd = sc.parallelize([(0, 5), (3, 8), (2, 6), (0, 8), (3, 8), (1, 3)])

# Aplicamos repartitionAndSortWithinPartitions con 2 particións
# A función de particionamento asigna partición segundo a paridade da clave (x % 2)
# O ordenamento será ascendente dentro de cada partición
rdd2 = rdd.repartitionAndSortWithinPartitions(
    numPartitions=2,
    partitionFunc=lambda x: x % 2,
    ascending=True
)

# Amosamos o contido de cada partición como listas para observar o reparto e a orde interna
resultado = rdd2.glom().collect()

print("RDD reparticionado e ordenado dentro das particións: ", resultado)

